In [4]:
import os
import subprocess
from pathlib import Path
import shutil
import glob
import zipfile

##Helper functions 
def find_latest_file(directory):
    """ Find the most recently created or modified file in a given directory."""
    files = glob.glob(f"{directory}/*")  # Get all files in the directory
    if not files:
        return None
    latest_file = max(files, key=os.path.getmtime)  # Get the most recently modified file
    return latest_file


def move_and_rename_file(source_dir, target_dir, params):
    """
    Move the latest generated file from the source directory to the target directory
    and rename it based on the given parameters.

    Parameters:
    - source_dir: The directory where the file is generated.
    - target_dir: The directory where the file should be moved.
    - params: A dictionary or list of parameters to include in the new file name.
    """
    # Find the latest file in the source directory
    latest_file = find_latest_file(source_dir)
    if not latest_file:
        print("Error: No file found to move.")
        return
    
    # Generate a new file name based on the parameters
    params_string = "_".join(map(str, params))  # Create a string from the parameters
    new_filename = f"simulation_{params_string}.txt"  # Custom name for the file
    target_path = Path(target_dir) / new_filename

    # Move and rename the file
    try:
        shutil.move(latest_file, target_path)
        print(f"File moved and renamed to: {target_path}")
    except Exception as e:
        print(f"Error while moving and renaming the file: {e}")


def run_simulation(command, results_dir="results", params=[]):
    """
    Run a simulation command and move its output file to the results directory.

    Parameters:
    - command: List of strings representing the CLI command to run the simulation.
    - results_dir: Directory where the results should be stored.
    - params: A list of parameters used for naming the output file.
    """
    # Ensure the results directory exists
    Path(results_dir).mkdir(parents=True, exist_ok=True)

    try:
        # Run the simulation command
        result = subprocess.run(
            command,
            text=True,           # Output as a string
            capture_output=True, # Capture stdout and stderr
            check=True           # Raise an error if the command fails
        )
        print("Simulation completed successfully.")

        # Move and rename the generated file
        move_and_rename_file(".", results_dir, params)
    
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while running the simulation: {e}")
        print(f"Standard output: {e.stdout}")
        print(f"Error output: {e.stderr}")


def zip_inner_loop_files(directory, zip_name, pattern='*.txt'):
    """
    Compress all files in a directory matching a given pattern into a single ZIP archive
    with ZIP64 support for large files or large archives.

    Parameters:
    - directory: The directory containing the files to zip.
    - zip_name: The name of the resulting ZIP archive.
    - pattern: Optional glob pattern to filter files (e.g., "*.txt").
    """
    # Ensure the ZIP file is saved inside the directory
    zip_path = Path(directory) / zip_name

    # Collect all files first to avoid deleting while iterating
    files_to_zip = [file for file in Path(directory).iterdir() if file.is_file() and (pattern is None or file.match(pattern))]

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED, allowZip64=True) as zipf:
        for file in files_to_zip:
            zipf.write(file, arcname=file.name)  # Add files without full path
            file.unlink()  # Delete the file after zipping

    print(f"All files matching pattern in '{directory}' have been zipped into: {zip_path}")


In [1]:
model = "delta"
regimes = ["Jump", "Flow", "No_Effect"]
input_distances = [60, 90, 120, 150]
params = {"Jump":
        {60: (0.628800, 1.329734),
        90: (0.515750, 2.341531),
        120: (0.515750, 2.341531),
        150: (0.515750, 2.341531)}, 
        "Flow":
        {60: (0.515750, 2.341531),
        90: (0.158750, 2.341531),
        120: (0.099250, 2.341531),
        150: (0.072475, 2.341531)}, 
        "No_Effect":
        {60: (0.010000, 2.341531),
        90: (0.015950, 2.326867),
        120: (0.015950, 2.326867),
        150: (0.015950, 2.341531)}}

ablation_distances = [10, 20, 30, 60, 80, 100, 120, 140, 160, 180]
num_ablated_neurons = [8, 16, 32, 64, 128]

In [4]:
params["No_Effect"][60][1]

2.341531

In [33]:
# Base case no need to zip
for regime in regimes:
    for input_distance in input_distances:
        results_dir = f"results/delta_damage/delta_damage_{input_distance}/{regime}"

        command = [
            "build/src/simuself",
            model,
            f"{input_distance}",
            f"{params[regime][input_distance][0]}",
            f"{params[regime][input_distance][1]}"
        ]
        labels = [f"{model}_{input_distance}", f"{regime}_{params[regime][input_distance][0]}_{params[regime][input_distance][1]}"]

        run_simulation(command, results_dir=results_dir, params=labels)

Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_60/Jump/simulation_delta_60_Jump_0.6288_1.329734.txt
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_90/Jump/simulation_delta_90_Jump_0.51575_2.341531.txt
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_120/Jump/simulation_delta_120_Jump_0.51575_2.341531.txt
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_150/Jump/simulation_delta_150_Jump_0.51575_2.341531.txt
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_60/Flow/simulation_delta_60_Flow_0.51575_2.341531.txt
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_90/Flow/simulation_delta_90_Flow_0.15875_2.341531.txt
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_1

In [29]:
# Damage Simulation Loop
for regime in regimes:
    for input_distance in input_distances:
        results_dir = f"results/delta_damage/delta_damage_{input_distance}/{regime}"
    
        for ablation_distance in ablation_distances:
        # Name the ZIP file for the current inner loop
            zip_name = f"sim_{model}_{input_distance}_damage_{regime}_{ablation_distance}.zip"

            for neurons in num_ablated_neurons:
                command = [
                    "build/src/simuself",
                    model,
                    f"{input_distance}",
                    f"{params[regime][input_distance][0]}",
                    f"{params[regime][input_distance][1]}",
                    "--damage",
                    f"{ablation_distance}",
                    f"{neurons}"
                ]
                labels = [f"{model}_{input_distance}", f"damage_{regime}_{ablation_distance}_{neurons}"]

                run_simulation(command, results_dir=results_dir, params=labels)
            
            # Once the inner loop completes, zip all generated files into the results directory
            zip_inner_loop_files(results_dir, zip_name)

Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_60/Jump/simulation_delta_60_damage_Jump_10_8.txt
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_60/Jump/simulation_delta_60_damage_Jump_10_16.txt
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_60/Jump/simulation_delta_60_damage_Jump_10_32.txt
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_60/Jump/simulation_delta_60_damage_Jump_10_64.txt
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_60/Jump/simulation_delta_60_damage_Jump_10_128.txt
All files matching pattern in 'results/delta_damage/delta_damage_60/Jump' have been zipped into: results/delta_damage/delta_damage_60/Jump/sim_delta_60_damage_Jump_10.zip
Simulation completed successfully.
File moved and renamed to: results/delta_damage/delta_damage_60/

In [44]:
# random damage simulation loop
for regime in regimes:
    for input_distance in input_distances:
        results_dir = f"results/delta_random/delta_random_{input_distance}/{regime}"
    


        
        zip_name = f"sim_{model}_{input_distance}_random_{regime}.zip"
        for neurons in num_ablated_neurons:
            
            command = [
                "ring_attractor/build/src/simuself",
                model,
                f"{input_distance}",
                f"{params[regime][input_distance][0]}",
                f"{params[regime][input_distance][1]}",
                "--random_damage",
                f"{neurons}"
            ]
            labels = [f"{model}_{input_distance}", f"random_damage_{regime}_{neurons}"]

            run_simulation(command, results_dir=results_dir, params=labels)
            
        # Once the inner loop completes, zip all generated files into the results directory
        zip_inner_loop_files(results_dir, zip_name)

Simulation completed successfully.
File moved and renamed to: results/delta_random/delta_random_60/Jump/simulation_delta_60_random_damage_Jump_8.txt
Simulation completed successfully.
File moved and renamed to: results/delta_random/delta_random_60/Jump/simulation_delta_60_random_damage_Jump_16.txt
Simulation completed successfully.
File moved and renamed to: results/delta_random/delta_random_60/Jump/simulation_delta_60_random_damage_Jump_32.txt
Simulation completed successfully.
File moved and renamed to: results/delta_random/delta_random_60/Jump/simulation_delta_60_random_damage_Jump_64.txt
Simulation completed successfully.
File moved and renamed to: results/delta_random/delta_random_60/Jump/simulation_delta_60_random_damage_Jump_128.txt
All files matching pattern in 'results/delta_random/delta_random_60/Jump' have been zipped into: results/delta_random/delta_random_60/Jump/sim_delta_60_random_Jump.zip
Simulation completed successfully.
File moved and renamed to: results/delta_random

### Scraps?

In [ ]:
#so what do we loop over here?
# we want to loop over the fact that the bump can be centered in a bunch of places relative to the input bump. say the input bump is at 120 degrees. Then we want to do some interval before and after. so say, 15, 30, 45, 90, etc. and then we want to varry how many neurons we knock out. from 8 to 248 (we can just throw away the ones that don't work)
model = 'delta'
input_distance = 90
amp = 0.158750
width = 2.341531
distances = range(10, 360, 15)
damaged_neurons = [8, 16, 32, 64, 128]

#damage simulation loop:

# Simulation Loop
results_dir = "results/delta_damage/delta_damage_90/No_Effect"
for distance in distances:
    # Name the ZIP file for the current inner loop
    zip_name = f"sim_{model}_{input_distance}_damage_{distance}.zip"

    for neurons in damaged_neurons:
        command = [
            "build/src/simuself",
            model,
            f"{input_distance}",
            f"{amp}",
            f"{width}",
            "--damage",
            f"{distance}",
            f"{neurons}"
        ]
        params = [f"{model}_{input_distance}", f"damage_{distance}_{neurons}"]

        run_simulation(command, results_dir=results_dir, params=params)
    
    # Once the inner loop completes, zip all generated files into the results directory
    zip_inner_loop_files(results_dir, zip_name)


In [ ]:
#so what do we loop over here?
# we want to loop over the fact that the bump can be centered in a bunch of places relative to the input bump. say the input bump is at 120 degrees. Then we want to do some interval before and after. so say, 15, 30, 45, 90, etc. and then we want to varry how many neurons we knock out. from 8 to 248 (we can just throw away the ones that don't work)
model = 'delta'
input_distance = 90
amp = 0.158750
width = 2.341531
distances = range(10, 360, 15)
damaged_neurons = [8, 16, 32, 64, 128]

#damage simulation loop:

# Simulation Loop
results_dir = "results/delta_damage/delta_damage_90/Flow"
for distance in distances:
    # Name the ZIP file for the current inner loop
    zip_name = f"sim_{model}_{input_distance}_damage_{distance}.zip"

    for neurons in damaged_neurons:
        command = [
            "build/src/simuself",
            model,
            f"{input_distance}",
            f"{amp}",
            f"{width}",
            "--damage",
            f"{distance}",
            f"{neurons}"
        ]
        params = [f"{model}_{input_distance}", f"damage_{distance}_{neurons}"]

        run_simulation(command, results_dir=results_dir, params=params)
    
    # Once the inner loop completes, zip all generated files into the results directory
    zip_inner_loop_files(results_dir, zip_name)


In [ ]:
#so what do we loop over here?
# we want to loop over the fact that the bump can be centered in a bunch of places relative to the input bump. say the input bump is at 120 degrees. Then we want to do some interval before and after. so say, 15, 30, 45, 90, etc. and then we want to varry how many neurons we knock out. from 8 to 248 (we can just throw away the ones that don't work)
model = 'delta'
input_distance = 90
amp = 0.515750
width = 2.341531
distances = range(10, 360, 15)
damaged_neurons = [8, 16, 32, 64, 128]

#damage simulation loop:

# Simulation Loop
results_dir = "results/delta_damage/delta_damage_90/Jump"
for distance in distances:
    # Name the ZIP file for the current inner loop
    zip_name = f"sim_{model}_{input_distance}_damage_{distance}.zip"

    for neurons in damaged_neurons:
        command = [
            "build/src/simuself",
            model,
            f"{input_distance}",
            f"{amp}",
            f"{width}",
            "--damage",
            f"{distance}",
            f"{neurons}"
        ]
        params = [f"{model}_{input_distance}", f"damage_{distance}_{neurons}"]

        run_simulation(command, results_dir=results_dir, params=params)
    
    # Once the inner loop completes, zip all generated files into the results directory
    zip_inner_loop_files(results_dir, zip_name)
